In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Importação de bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# O modulo wordcloud não é nativo, descomente a linha abaixo se nao a tiver instalado
#! pip install wordcloud

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Avaliação Bimestral 1
## Analise Estatística Para Ciência de Dados
### Especialização em Ciência de Dados
#### Instituto Federal de Educação, Pesquisa, Ciência e Tecnologia de São Paulo (IFSP) - Câmpus Campinas

Este projeto foi um trabalho da pós-graduação: Especialização em Ciência de Dados, na disciplina de Análise Estatística para Ciência de Dados, para composição de atividades da disciplina no semestre. O trabalho consiste em análise exploratória de um conjunto de dados referentes a filmes e suas respectivas avaliações (e comentários) feitos por usuários.

Por: 

**Carlos Danilo Tomé** 

**Lucas Galdino de Camargo** 


## **1. Especificação**

Nesta atividade, cada dupla deverá realizar uma análise exploratória para o conjunto de dados
Movie Lens (small): https://grouplens.org/datasets/movielens, a partir de um conjunto de perguntas e
hipóteses pré-estabelecidas e propostas pela dupla.

Seguem os critérios a serem avaliados. Cada critério tem um conjunto de pontos que servirão como um guia para seu desenvolvimento.

## **2. Descrição da base de dados escolhida:**
    
**2.1 O que significa cada linha de cada arquivo?**

*Resposta:*

No conjunto de dados Movie Lens temos 4 arquivos referentes a diferentes caracteristicas e informações sobre os dados, são elas: 

**ratings.csv**: O arquivo contém a cada linha uma avaliação de um filme por um usuário, com 100836 observações. 

**tags.csv**: O arquivo contém a cada linha uma "tag" atribuída a um filme por um usuário, com 3683 observações.

**movies.csv**: O arquivo contém a cada linha informações sobre um filme, com 9742 observações.

**links.csv**: O arquivo contém identificadores associados a cada filme e pode ser utilizado para utilizar informações de outras fontes, com 9742 observações.

**2.2 Quais são os principais atributos (colunas) e seus tipos?**

*Resposta:* 

**ratings.csv**:

- userid (ID do usuário que fez a avaliação, tipo int)
- movieid (ID do filme avaliado, tipo int)
- rating (avaliação do filme, em escala de 5 estrelas, com incrementos de 0.5 entre as avaliações, varia de 0.5 estrelas a 5.0 estrelas, tipo float) 
- timestamp (representa os segundos desde a meia noite de 01 de janeiro de 1970, Coordinated Universal Time (UTC), tipo int)

**tags.csv**: 

- userid (ID do usuário que fez a avaliação, tipo int)
- movieid (ID do filme avaliado, tipo int)
- tag ("tag" atribuída pelo usuário ao filme, geralmente uma palavra simples ou frase curta, mas o significado, valor, e propósito de uma tag particular é determinada por cada usuário, que tem liberdade de digitar a tag que quiser, tipo object)
- timestamp (representa os segundos desde a meia noite de 01 de janeiro de 1970, Coordinated Universal Time (UTC), tipo int)

**movies.csv**: 

- movieid (ID do filme avaliado, tipo int)
- title (Título do filme, acompanhado do ano entre parêntesis, tipo object) erros ou inconsistências podem ocorrer nesses títulos
- genres (Gêneros atribuídos ao filme, tipo object, segundo a lista: 
    Action,
    Adventure,
    Animation,
    Children's,
    Comedy,
    Crime,
    Documentary,
    Drama,
    Fantasy,
    Film-Noir,
    Horror,
    Musical,
    Mystery,
    Romance,
    Sci-Fi,
    Thriller,
    War,
    Western,
    (no genres listed))

**links.csv**: 

- movieid (ID do filme usado em <https://movielens.org>, por exemplo: o filme "Toy Story" tem o link <https://movielens.org/movies/1>, tipo int)
- imdbid (ID do filme usado em <http://www.imdb.com>, por exemplo: o filme "Toy Story" tem o link <http://www.imdb.com/title/tt0114709/>, tipo int)

- tmdbid (ID do filme usado em <https://www.themoviedb.org>, por exemplo: o filme "Toy Story" tem o link <https://www.themoviedb.org/movie/862>, tipo float)

In [ ]:
# Importando as bases de dados

# ratings.csv
df_ratings = pd.read_csv('../input/movielens-latest-small/ratings.csv',encoding = 'utf-8')

# tags.csv
df_tags = pd.read_csv('../input/movielens-latest-small/tags.csv',encoding = 'utf-8')

# movies.csv
df_movies = pd.read_csv('../input/movielens-latest-small/movies.csv',encoding = 'utf-8')

# links.csv
df_links = pd.read_csv('../input/movielens-latest-small/links.csv',encoding = 'utf-8')

## **3. Preparação da base de dados:**


- Teve dificuldades para preparar o dataset para uso (p. ex., nomes ruins para as colunas, arquivos dos datasets não possuíam boa organização)?

R: Os dataframes em geral precisam de algumas adaptações para responder ao menos as perguntas iniciais propostas, além disso só foi necessário uma transformação do tipo de dados nas colunas referentes à dados do tipo **Data**. Os demais tratamentos e feature engineering foram realizados de forma à aumentar a quantidade de informações relevantes sobre as bases de dados e o contexto da analise exploratória. Também foi necessária unificar algumas informações realizando operações "merge" para trazer informações de tabelas diferentes ao dataframe principal da nossa análise **df_movies**.

- Aplicou algum pré-processamento dos dados?
    - Outliers/ruídos, dados faltantes, conversão de tipos de dados, etc.

R: Foram aplicados 3 tipos de tratamento de dados distintos, **Conversão do tipo do dado**, **Criação de colunas** e **Tratamento de dados faltantes**, esses tratamentos foram sistematizados por meio de funções que aplicadas na ordem correta trazem todas as informações necessárias para a analise exploratoria do contexto proposto. Os tratamentos e as respectivas funções estão explicitadas abaixo;

### Descrição dos tratamentos

##### Conversão de tipo de dados


- **1. Transformar as datas**

Esse tratamento foi utilizado nos dataframes df_rating e df_tags. O formato da variável 'timestamp' nestes dataframes (em segundos, desde a meia noite de 01 de janeiro de 1970, Coordinated Universal Time (UTC), tipo int) não é apropriado para manipulação de dados em formato de data, demandando transformação.


##### Criação de Colunas

- **2. Retirar do Titulo a data de lançamento do filme**

Esse tratamento foi utilizado no dataframe df_movies, inicialmente a intenção é capturar a informação da data de lançamento. Essa aplicação está identificada como a função **"convert_title_to_year"**.


- **3. Criação das colunas: 'media_ratings', 'ratings_count' e 'user_count_rating'**

Esse tratamento foi aplicado no dataframe df_movies, neste tratamento estamos adicionando três colunas ao dataframe principal df_movies referente à informações de ratings por filme. As colunas são **'media_ratings'** (média de rating por filme),  **'ratings_count'** (quantidade de rating registrados por filme) e **'user_count_rating'** (quantidade de usuarios distintos que fizeram uma avaliação por filme). 

Esta aplicação está identificada pela função **'calc_media_contagem_rating'**.


- **4. Criação das colunas: tags_count e user_count_tag**

Esse tratamento foi aplicado no dataframe df_movies, neste tratamento estamos adicionando duas colunas ao dataframe principal df_movies referente a informações de tags por filme. As colunas são **'tags_count'** (quantidade de tags registradas por filme) e **'user_count_tag'** (quantidade de usuarios distintos que marcaram uma tag por filme). 

Esta aplicação esta identificada pela função **'calc_contagem_tags'**.


- **5. Criação das colunas: Booleano por tipo de genero e quantidade de generos**

Esse tratamento foi aplicado no dataframe df_movies, neste tratamento estamos adicionando vinte colunas ao dataframe principal df_movies referente a informações de genero e a quantidade de generos por filme. São 19 colunas de formato booleano adicionadas referentes à classificação do filme, se o filme estiver identificado com algum genero ele é marcado como 1, caso contrario 0, para todos os generos **['Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','(no genres listed)']** e por fim adiciona também a coluna **"qtde_generos"**, referente a soma de generos classificados por filme.

Esta aplicação esta identificada pela função **'colunas_genero'**.

- **6. Criação da coluna: 'Distancia em Anos'**

Neste tratamento estamos adicionando uma coluna aos dataframe df_ratings e df_tags referente a distancia em anos do lançamento do filme e a data de pubicação do rating e/ou tag. A coluna **'Distancia em Anos'** é calculada como a diferenteça, em anos, da data de lançamento do filme até a publicação da observação.

Esta aplicação esta identificada pela função **'colunas_diferenca_anos'**.


##### Dados Faltantes

Após esses tratamentos e unificação de bases e informações temos valores nulos em algumas das colunas que foram calculadas conforme instruções acima. Uma das mais importantes são a quantidade de filmes sem rating ou tag, filmes sem alguma observação nos dataframes auxiliares df_ratings e df_tags geram valores faltantes nas colunas calculadas, são elas:


    - media_ratings, ratings_count e user_count_rating:  
    
    Existem 18 valores faltantes nestas colunas referentes à filmes sem nenhuma avaliação por parte dos usuarios. Ação: Para uma analise exploratoria inicial não serão retiradas essas informações faltantes do dataframe principal devida a perda de informação, esta é uma informação adicional e calculada, quando gerados visualizações explorando essas informações os valores faltantes são desconsiderados.

    - tags_count e user_count_tag: 
    
    Existem 8170 valores faltantes nestas colunas referentes à filmes sem nenhuma tag gerada por parte dos usuarios. Ação: Para uma analise exploratoria inicial não serão retiradas essas informações faltantes do dataframe principal devida a perda de informação, esta é uma informação adicional e calculada, quando gerados visualizações explorando essas informações os valores faltantes são desconsiderados.

    - ano: 
    
    Existem 12 valores faltantes nestas colunas referentes à filmes que não tinham, como os demais filmes por padrão, o ano relativo ao lançamento do filme entre parênteses. Ação: Os filmes que não tiveram ano de lancamento identificado após a fase de tratamento de dados serão retirados da base, pois esta informação é primordial para as analises discutidas nas perguntas abaixo.

In [ ]:
def convert_date(df, coluna):

    # Converte a quantidade de segundos para Data com Minutos
    df[coluna] = pd.to_datetime(df[coluna], unit='s')

    # Transforma no formato datetime 64
    df[coluna] = pd.to_datetime(df[coluna], format='%Y-%m-%d')

    # Cria a coluna - 'ano de avaliação'
    df['Ano de Avaliação'] = df.timestamp.dt.year
    return df;

def convert_title_to_year(df):

    # Retira todos os espaços da coluna "title"
    df['title']= df['title'].str.rstrip()

    # Cria coluna 'ano' com os últimos caracteres da coluna "title"
    df['ano'] = df.title.str.slice(-5,-1,1)

    # Transforma a coluna "ano" em inteiro
    # Com excessão dos filmes que não tenham a informação do ano entre parenteses
    # A opção 'coerce' faz com que valores não númericos sejam considerados vazios
    df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

    return df;

def calc_media_contagem_rating(df):

    # Agrupa as notas por movieId e calcula a respectiva média de avaliação, contagem de avaliações e contagem de usuarios que fizeram avaliações.
    temp = df_ratings.groupby(['movieId']).apply(lambda x: pd.Series(dict(media_ratings         =  (round(x.rating.mean(),1))
                                                                                ,ratings_count  =  ((x.rating.count()))
                                                                                ,user_count_rating  =  ((x.userId.nunique()))
                                                                                ))).reset_index()

    df = pd.merge(df, temp, how = 'left', on = ['movieId'])     

    return df;    

def calc_contagem_tags(df):

    # Agrupa por filme e calcula duas variaveis: Contagem de tags e contagem de usuarios que fizeram avaliacoes 
    temp = df_tags.groupby(['movieId']).apply(lambda x: pd.Series(dict(tags_count  =  ((x.tag.count()))
                                                                    ,user_count_tag  =  ((x.userId.nunique()))
                                                                    ))).reset_index()

    df = pd.merge(df, temp, how = 'left', on = ['movieId'])                   
    return df;   

# Função que adiciona na tabela quais respectivos generos são atribuidos para cada filme
def colunas_genero(df):

    # Cria coluna com a quantidade de generos respectivo a cada filme
    df['qtde_generos'] = df.apply(lambda x : len(x['genres'].split("|")),axis=1)

    # Lista de todos os generos de filmes possíveis
    lista = ['Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir'
                ,'Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','(no genres listed)']

    # Função que atribui 1 caso o a string do gênero contenha o valor da respectiva coluna
    def atribui_um(coluna, genre):
        if coluna in genre:
            return 1
        else:
            return 0 
            
    # Cria uma coluna para cada um dos generos possiveis
    for column in lista:

        # Divide a coluna "genres" pelo sinal "|" e atribui o valor um caso o genero esteja dentro dos valores para cada filme
        df[column] = df.apply(lambda x: atribui_um(column,x['genres'].split(sep="|",maxsplit=10)),axis=1)
    
    return df;

def colunas_diferenca_anos(df):
    # Busca a informação de ano de lançamento do filme na tabela df_movies (aproveitando para já trazer o título do filme)
    df = pd.merge(df, df_movies[['movieId', 'ano', 'title']], how = 'left', on=['movieId'])

    # Cria uma coluna ilustrativa a distância em anos do lançamento do filme até o ano da avaliação
    df['Distancia em Anos'] = df['Ano de Avaliação'] - df['ano'] 

    # Retira valores discrepantes em df_ratings
    df = df[df['Distancia em Anos'] > -1]

    return df;

In [ ]:
# Convertendo o tipo de data no Dataframe df_ratings 
df_ratings = convert_date(df_ratings, 'timestamp')

# Convertendo o tipo de data no Dataframe df_tags 
df_tags = convert_date(df_tags, 'timestamp')

# Criando a coluna ano no Dataframe df_movies
df_movies = convert_title_to_year(df_movies)

# Adiciona colunas com a contagem e média de Ratings
df_movies = calc_media_contagem_rating(df_movies)

# Adiciona as colunas de contagem de tags
df_movies = calc_contagem_tags(df_movies)

# Adiciona informação de genero para cada filme
df_movies = colunas_genero(df_movies)

# Adiciona coluna referente a diferença de Anos entre a avaliação e o lançamento do filme na tabela df_rating
df_ratings = colunas_diferenca_anos(df_ratings)

# Adiciona coluna referente a diferença de Anos entre a avaliação e o lançamento do filme na tabela df_tags
df_tags = colunas_diferenca_anos(df_tags)

# Excluindo linhas em que a informação de Ano do lançamento do filme está faltando
df_movies.dropna(subset = ['ano'], inplace = True)

## **4. Análise Exploratória:**

- #### Pergunta 1 - Existe alguma correlação entre o ano de lançamento do filme e sua avaliação média (média de ratings)?

R:

Como podemos ver pela matriz de correlação abaixo, há uma correlação negativa muito fraca entre o ano de lançamento do filme e sua avaliação média (aproximadamente -0.1). Contudo como podemos visualizar no segundo gráfico abaixo, filmes anteriores ao ano de 1970 tem menos pontos entre as avaliações baixas (1 e 2), enquanto nos anos posteriores as avaliações estão mais dispersas recebendo mais observações em filmes nota 5, 2 e 1 do que os filmes antigos. Isso é devido à volumetria nas quantidades de avaliações de filmes: filmes lançados até 1970 não possuem volume tão grande de avaliações como filmes a partir desta década e, por isso, a dispersão de dados aumenta conforme a quantidade de avaliações.

In [ ]:
# Compute the correlation matrix
corr = round(df_movies[['media_ratings', 'ano', 'ratings_count', 'tags_count', 'qtde_generos']].corr(),1)

# Plota a matriz de correlação
sns.heatmap(corr, cmap="YlGnBu", vmin=-0.5, vmax=0.5, annot=True);

In [ ]:
# Grafico de dispersão de Ano de lançamento por média de rating
fig = px.scatter(df_movies, x="ano", y="media_ratings",hover_data=['media_ratings'])

fig.update_layout(
    title='Avaliação Média dos filmes por Ano de Lançamento',
    xaxis_title="Ano de Lançamento",
    yaxis_title="Média de Avaliação",
        template='plotly_white')

fig.show()

In [ ]:
# Cria o objeto da figura do gráfico
fig = px.histogram(df_ratings,x="ano",  marginal="violin" )

# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Quantidade de avaliações por Ano de lançamento do filme',
    xaxis_title="Ano de Lançamento",
    yaxis_title="Quantidade de Avaliações",
    template='plotly_white')

# Mostra o gráfico
fig.show()

- #### Pergunta 2 - Qual o grupo mais disperso, o de usuários que avaliaram ou o de usuários que rotularam filmes?

R:

Como podemos ver no próximo gráfico da série, em que no eixo x temos a distribuição da quantidade de avaliações feitas por usuário (bem como a quantidade de atribuição de tags em filmes, informação que vem empilhadada), e no eixo y nós temos a quantidade de usuários, podemos observar que:

1) Há muito mais usuários que avaliaram filmes do que usuários que atribuíram tags;

2) Podemos ver que usuários que atribuíram tags estão bem concentrados em um range de até 49 tags atribuídas em filmes, tendo muitos poucos usuários com mais de 50 tags atribuídas em filmes;

3) Com isso, concluímos que a dispersão na quantidade de avaliações feitas por usuário é muito maior para usuários que avaliaram filmes do que os usuários que atribuíram tags;

In [ ]:
# Agrupa as tags mais rotuladas por quantidade de filmes
temp_ratings = df_ratings.groupby(['userId']).count().sort_values(by='userId', ascending = False)

# Agrupa as tags mais rotuladas por quantidade de filmes
temp_tags = df_tags.groupby(['userId']).count().sort_values(by='userId', ascending = False)

# Cria uma coluna temporaria para marcar como historico a base referente a avaliação
temp_ratings['flag'] = 'Atribuiu Nota'
temp_tags['flag'] = 'Atribuiu Rotulo'

# Pilha a quantidade de avaliações feita por usuario em cada tipo
temp_final = pd.concat([temp_ratings[['movieId','flag']], temp_tags[['movieId','flag']]])
temp_final.rename(columns = {'movieId': 'Contagem de Avaliações'}, inplace = True)

# Plotando grafico sobre a dispersão da quantidade de avaliações e rotulos feitos por cada usuario

fig = px.histogram(temp_final, x="Contagem de Avaliações",color = 'flag', marginal="violin" )

fig.update_layout(
    title='Distribuição da quantidade de avaliações feitas por cada usuario',
    xaxis_title="Quantidade de Avaliações",
    yaxis_title="Quantidade de Usuários",
        template='plotly_white')

fig.show()

- #### Pergunta 3 - Qual a categoria rotulada mais frequente?

R:

Como podemos ver pelo gráfico abaixo o rótulo mais frequente nos filmes é "IN NETFLIX QUEUE".

Isso nos faz pensar que o algoritmo da Netflix para recomendação de títulos funciona bem para usuários que atribuem tags a filmes, já que esta é de longe a tag mais mencionada pelos usuários, indicando que eles possivelmente assistiram ao título com base no sistema (algoritmo) de recomendação de filmes da Netflix.

In [ ]:
# Cria uma coluna com a string toda em UPPERCASE para diminuir eventuais diferenças básicas
df_tags['tag_upper'] = df_tags.tag.str.upper()

# Agrupa as tags mais rotuladas por quantidade de filmes
temp = df_tags.groupby(['tag_upper']).count().sort_values(by='movieId', ascending = False)

# Plota o gráfico de número de ocorrencias das 20 tags mais comuns
fig = go.Figure(data=[go.Bar(
            x=temp[0:20].index, y=temp[0:20].userId,
            text=temp[0:20].userId,
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Os 20 tags mais mencionadas pelos usuarios',
    xaxis_title="Tag",
    yaxis_title="Quantidade de Menções",
        template='plotly_white')

fig.show()

- #### Pergunta 4 - Como estão distribuídas as avaliações ao longo do tempo?

R:

Podemos ver a partir do boxplot abaixo que filmes lançados até 1970 são outliers quando avaliamos a quantidade de avaliações recebidas, o 1º quartil se encontra no ano de 1990, a Mediana está no ano de 1997, o 3º quartil é o ano de 2003 e os dados são referentes a filmes lançados até 2018.

Com isso, temos que 50% das nossas avaliações foram feitas para filmes lançados de 1990 a 2003.

Na sequência abaixo (2º gráfico a seguir), trouxemos os boxplots das avaliações médias dos filmes cujo ano de lançamento seja maior que 1970 (já que os filmes antes disso possuem pouquíssimas avaliações, e são outliers nesse sentido).

Deste gráfico conseguimos ver que as avaliações médias de filmes ao longo do tempo está sempre entre 3 e 4 (e a mediana das avaliações médias dos filmes por ano esteve sempre nesse intervalo), além disso, já sabíamos da primeira pergunta que praticamente não existe correlação entre a avaliação média do filme e seu ano de lançamento.

In [ ]:
# Plota o boxplot das avaliações por ano de lançamento do filme
fig = px.box(df_ratings, x="ano")
# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title ='Distribuição das avaliações por ano de lançamento do filme',
    xaxis_title="Ano de lançamento")
fig.show()

# Filtra apenas filmes lançados a partir de 1971
temp = df_movies.loc[(df_movies['ano'] > 1970)]
# Plota a distribuição das avaliações médias dos filmes, por ano, para os filmes lançados a partir de 1971
fig = px.box(temp, x="ano", y="media_ratings", title ='Distribuição das Avaliações Médias dos filmes por Ano de Lançamento' )
fig.show()

- #### Pergunta 5 - Como estão distribuídas as avaliações a partir da data de lançamento?

R:

A seguir, segue algumas visualizações para ajudar a entender o que ocorre com as avaliações dos filmes conforme a 'Idade' dos filmes vai aumentando ao longo do tempo.

Primeiro, um gráfico de barras (combinado com um violin plot) com a distribuição das avaliações feitas considerando a 'Idade' do filme no momento da avaliação, ou seja, a diferença em anos entre o ano da avaliação e o ano de lançamento do filme. 

Na sequência, um boxplot com a distribuição das avaliações considerando a 'Idade' do filme no momento da avaliação.

Deste(s) gráfico(s) podemos ver que:

    - 1º quartil é em 3 anos;
    - Mediana é em 9 anos;
    - 3º quartil é em 18 anos;
    - Filmes com mais de 40 anos representam muito pouco no nosso conjunto de filmes com avaliação.

Tendo conhecimento disto, nós trouxemos os boxplots das avaliações dos filmes considerando a 'Idade' do filme no momento da avaliação, e logo abaixo nós trouxemos a mesma visão, porém considerando apenas as avaliações para filmes com no máximo 60 anos desde seu lançamento no momento da avaliação. Com esses boxplots nós conseguimos notar um comportamento muito curioso:

    - As distribuições das avaliações de filmes de até 23 anos (o que corresponde a mais de 75% das nossas avaliações) é praticamente a mesma, tendo o 1º quartil na nota 3, mediana em 3.5 e 3º quartil em 4 (com exceção do ano de lançamento do filme, que eleva a mediana para 4, enquanto os quartis se mantém);
    - Outro padrão observado foi para filmes avaliados com pelo menos 24 anos desde seu lançamento, que possuem 1º quartil variando entre 3 e 3.5, mediana em 4 e 3º quartil variando entre 4 e 5.

Dadas essas últimas observações, pudemos perceber que há praticamente 2 comportamentos nas avaliações de filmes considerando a 'Idade' do filme no momento da avaliação:

    - Filmes de pelo menos 24 anos possuem avaliações melhores (e mais dispersas) do que as avaliações para filmes de até 23 anos desde seu lançamento.

In [ ]:
# Cria o objeto da figura do gráfico
fig = px.histogram(df_ratings,x="Distancia em Anos",  marginal="violin" )

# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Distribuição das avaliações de filmes por tempo em anos entre o lançamento e a avaliação',
    xaxis_title="'Idade' do filme no momento da avaliação",
    yaxis_title="Quantidade de Avaliações",
    template='plotly_white')

# Mostra o gráfico
fig.show()

# Plota o boxplot da Distância em anos entre as avaliações dos filmes e seus respectivos anos de lançamento ('Idade' do filme no momento da avaliação)
fig = px.box(df_ratings, x="Distancia em Anos")
# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Distribuição das avaliações considerando a "Idade" do filme no momento da avaliação',
    xaxis_title="'Idade' do filme no momento da avaliação",
    yaxis_title="Quantidade de Avaliações",
    template='plotly_white')

fig.show()

In [ ]:
# Plota boxplots das avaliações considerando a 'Idade' do filme no momento da avaliação.
fig = px.box(df_ratings, x="Distancia em Anos", y="rating")
# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Distribuição das avaliações considerando a "Idade" do filme no momento da avaliação',
    xaxis_title="'Idade' do filme no momento da avaliação",
    yaxis_title="Distribuição de Ratings",
    template='plotly_white')
fig.show()

- #### Pergunta 6 - Como estão dispostos os gêneros de filmes por quantidade de avaliação e média de ratings?

R:

Segue a lista dos 5 gêneros com a maior quantidade de avaliações, e suas respectivas médias de avaliação:

    - 1) Drama, avaliações: 41,9k (1º), média: 3,66 (4º)
    - 2) Comedy, avaliações: 39,1k (2º), média: 3,38 (17º)
    - 3) Action, avaliações: 30,6k (3º), média: 3,45 (15º)
    - 4) Thriller, avaliações: 26,4k (4º), média: 3,49 (13º)
    - 5) Adventure, avaliações: 24,2k (5º), média: 3,51 (11º)

Dentre os 5 gêneros com a maior quantidade de avaliações, apenas 1 aparece também entre a lista de 5 gêneros com a melhor média de avaliações: Drama, 1º em quantidade de avaliações e 4º em média de avaliações.

Segue também a lista dos 5 gêneros com melhor média de avaliação, e suas respectivas quantidades de avaliação:

    - 1) Film-Noir, média 3,92 (1º), avaliações: 0,9k (17º)
    - 2) War, média 3,81 (2º), avaliações: 4,9k (13º)
    - 3) Documentary, média: 3,80 (3º), avaliações: 1,2k (16º)
    - 4) Drama, média: 3,66 (4º), avaliações: 41,9k (1º)
    - 5) Crime, média: 3,66 (5º), avaliações: 16,7 (8º)

Outro Gênero que chama atenção é Crime, que aparece em 5º na lista de melhores médias de avaliação, e em 8º na lista dos que tiveram mais avaliações.

Como podemos ver, entre os gêneros com melhor média, são recorrentes aqueles que estão entre os gêneros com menos avaliações. 

Além disso, a quantidade de avaliações por gênero não parece nos dizer muito quanto à avaliação média, já que apenas Drama aparece no Top 5 de quantidade de avaliações e avaliação média, mas parece nos dizer que podemos esperar mais filmes de Drama, Comédia e Ação sendo lançados do que Documentários, Film-Noir ou Western. Além disso, parece fazer sentido que o gênero Children's não tenha avaliações, já que não devemos esperar crianças entre os avaliadores (e comentaristas) de filmes.

In [ ]:
# Busca a informação de gêneros do filme
df_ratings = pd.merge(df_ratings, df_movies[['movieId','Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir'
                    ,'Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','(no genres listed)','qtde_generos']], how = 'left', on=['movieId'])

# lista com todos os gêneros de filmes
lista_generos = ['Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir'
                    ,'Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','(no genres listed)']

media, qntd_avaliacoes = [],[]

for coluna in lista_generos:

    # Cria um filtro para genero do filme
    temp = df_movies[df_movies[coluna] == 1].copy()
    temp_2 = df_ratings[df_ratings[coluna] == 1].copy()

    # Atribui a variavel qntd_avaliacoes  a quantidade de avaliações para filmes classificados pelo genero iterado
    qntd_avaliacoes.append(temp['ratings_count'].sum()) 

    # Atribui a variavel media_ratings com a média de ratings para filmes classificados pelo genero iterado
    media.append(round(temp_2['rating'].mean(), 2)) 

df_temp = pd.DataFrame({'Generos':lista_generos,
                        'Quantidade de Avaliações': qntd_avaliacoes,
                        'Media de Ratings':media })

In [ ]:
df_temp = df_temp.sort_values(by='Quantidade de Avaliações',ascending=False)

#Plota a quantidade de avaliações por gênero
fig = go.Figure(data=[go.Bar(
            x=df_temp.Generos, y=df_temp['Quantidade de Avaliações'],
            text=df_temp['Quantidade de Avaliações'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Quantidade de Avaliações por Gênero',
    xaxis_title="Gênero",
    yaxis_title="Quantidade de Avaliações",
        template='plotly_white')

fig.show()

df_temp = df_temp.sort_values(by='Media de Ratings',ascending=False)

#Plota a avaliação média por gênero
fig = go.Figure(data=[go.Bar(
            x=df_temp.Generos, y=df_temp['Media de Ratings'],
            text=df_temp['Media de Ratings'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Média das Avaliações por Gênero',
    xaxis_title="Gênero",
    yaxis_title="Rating",
        template='plotly_white')

fig.show()

- #### Pergunta 7 - Qual filme teve maior quantidade de rótulos? E quais foram os rótulos mais atribuídos? E para o segundo filme com mais rótulos? E o terceiro? 

R:

Como podemos ver no gráfico abaixo, os filmes que geraram maior engajamento dentre as classificações de filmes (tags) que temos disponíveis foram: 

    - 1º) Pulp Fiction (1994) - 181 tags atribuídas

    - 2º) Fight Club (1999) - 54 tags atribuídas

    - 3º) 2001: A Space Odyssey (1968) - 41 tags atribuídas

Para os demais filmes, a quantidade de tags atribuídas ao filme é sempre menor que 40.

Uma vez que Pulp Fiction tem uma quantidade de tags que é muito superior aos demais filmes, gostaríamos de saber quais foram as tags (ou palavras) mais encontradas quando se fala neste filme.

Por curiosidade, gostaríamos de saber também quais foram as tags (ou palavras) mais atribuídas para Fight Club e 2001: A Space Odyssey.

Abaixo há 3 mapas de palavras ("wordclouds"), com as palavras mais recorrentes nas tags atribuídas aos top 3 filmes com mais tags atribuídas.

Para Pulp Fiction (1994), algumas das palavras mais recorrentes foram:

    - Violencer;
    - Strong;
    - Violent;
    - Disturbing;
    - Great Dialogue;
    - Linear Cult;
    - Good.

Para Fight Club (1999), algumas das palavras mais recorrentes foram:

    - Dark;
    - Comedy;
    - Psychology;
    - Thought;
    - Ending Dark;
    - Provoking Twist.

Finalmente, para 2001: A Space Odyssey (1968) algumas das palavras mais recorrentes foram:

    - Effects;
    - Space;
    - Alliens.

In [ ]:
# Agrupa para sabermos quais filmes tiveram a maior quantidade de tags atribuídas
temp = df_tags.groupby(['title']).count().sort_values(by='movieId', ascending = False)

#Plota o gráfico de dos 20 filmes com mais tags atribuídas
fig = go.Figure(data=[go.Bar(
            x=temp[0:20].index, y=temp[0:20].userId,
            text=temp[0:20].userId,
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Os 20 filmes com mais tags',
    xaxis_title="Filme",
    yaxis_title="Quantidade de Tags",
        template='plotly_white')

fig.show()

In [ ]:
# Utilizando lista de stopwords já disponíveis na biblioteca wordcloud
stopwords = set(STOPWORDS)

def word_cloud_f(df,movie_name):
    # Filtra apenas o filme de interesse
    top_tags = df.loc[(df['title'] == movie_name)]

    # Junta todas as tags atribuídas
    all_tags = "".join(tag for tag in top_tags['tag_upper'])

    # Vamos ver a quantidade de palavras atribuídas por todas as tags para o filme selecionado
    print("Total de palavras: {}".format(len(all_tags)))

    # Gerando uma wordcloud
    wordcloud = WordCloud(stopwords=stopwords,
                        background_color = "black",
                        width=1600,
                        height=800).generate(all_tags)

    # Mostrando a nossa wordcloud
    fig, ax = plt.subplots(figsize=(10,6))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_axis_off()

    plt.imshow(wordcloud)

In [ ]:
#Plotando a nossa wordcloud para o filme que mais teve tags atribuídas (de longe): Pulp Fiction (1994)
word_cloud_f(df_tags,'Pulp Fiction (1994)')

In [ ]:
#Plotando a nossa wordcloud para o 2º filme com mais tags atribuídas: Fight Club (1999)
word_cloud_f(df_tags,'Fight Club (1999)')

In [ ]:
#Plotando a nossa wordcloud para o 3º filme com mais tags atribuídas: 2001: A Space Odyssey (1968)
word_cloud_f(df_tags,'2001: A Space Odyssey (1968)')

- #### Pergunta 8 - Quais filmes tiveram mais avaliações? E quais se sairam melhores?

R:

A seguir, nós trouxemos os top 20 filmes com melhor avaliação média (porém, todos ficaram com avaliação média em 5, e nenhum aparece entre os 20 filmes com a maior quantidade de avaliações).

Trouxemos também os top 20 filmes com a maior quantidade de avaliações.

Logo em seguida, trouxemos os top 20 filmes com os melhores ratings médios dentre os filmes que tiveram mais de 50 avaliações.

Com isso, segue abaixo alguns dos filmes que mais se destacaram, considerando tanto a quantidade de avaliações recebidas, quanto ao rating médio:

    - Shawshank Redemptions, The (1994) - 4.4 de Rating Médio (1º entre os filmes com pelo menos 51 avaliações) e 317 avaliações (2º Geral)
    - Pulp Fiction (1994) - 4.2 de Rating Médio (11º entre os filmes com pelo menos 51 avaliações) e 307 avaliações (3º Geral), além disso, vimos acima que é também o filme que mais teve atribuição de tags.
    - Forest Gump (1994) - 4.2 de Rating Médio (15º entre os filmes com pelo menos 51 avaliações) e 329 avaliações (1º Geral)
    - Fight Club (1999) - 4.3 de Rating Médio (3º entre os filmes com pelo menos 51 avaliações) e 218 avaliações (11º Geral)
    - Star Wars: Episode IV - A New Hope (1977) - 4.2 de Rating Médio (9º entre os filmes com pelo menos 51 avaliações) e 251 avaliações (6º Geral)

Uma outra observação relevante é que os filmes melhoras avaliados em média também estão entre os filmes que mais tem avaliação, o que levante uma hipótese:

- Existe uma correlação clara entre quantidade de Avaliações e a Média de Avaliações?

In [ ]:
df_temp = df_movies.sort_values(by='media_ratings',ascending=False)

df_temp = df_temp [0:20]

# Plota o gráfico dos 20 filmes de melhor rating médio
fig = go.Figure(data=[go.Bar(
            x=df_temp.title, y=df_temp['media_ratings'],
            text=df_temp['media_ratings'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Top 20 Filmes com melhores Avaliações Médias',
    xaxis_title="Genero",
    yaxis_title="Rating Médio",
        template='plotly_white')

fig.show()


df_temp = df_movies.sort_values(by='ratings_count',ascending=False)

df_temp = df_temp [0:20]

# Plota o gráfico de número de avaliações dos 20 filmes com a maior quantidade de avaliações
fig = go.Figure(data=[go.Bar(
            x=df_temp.title, y=df_temp['ratings_count'],
            text=df_temp['ratings_count'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Top 20 Filmes mais avaliados',
    xaxis_title="Filme",
    yaxis_title="Quantidade de Avaliações",
        template='plotly_white')

fig.show()

In [ ]:
# Refazendo os top 20 filmes com melhor rating médio, considerando apenas os que tiveram mais de 50 avaliações
df_temp = df_movies[df_movies['ratings_count'] > 50].sort_values(by='media_ratings',ascending=False)

df_temp = df_temp [0:20]

#Plota o gráfico dos 20 filmes de melhor rating médio, e mais de 50 avaliações
fig = go.Figure(data=[go.Bar(
            x=df_temp.title, y=df_temp['media_ratings'],
            text=df_temp['media_ratings'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
fig.update_layout(
    title='Top 20 Filmes com melhores Avaliações Médias (e avaliações totais > 50)',
    xaxis_title="Gênero",
    yaxis_title="Rating Médio",
        template='plotly_white')

fig.show()

- #### Pergunta 9 - Existe uma correlação clara entre quantidade de Avaliações e a Média de Avaliações?

R: Podemos ver pelo gráfico abaixo (avaliação média x quantidade de avaliações) que filmes com poucas avaliações (até 50), possuem uma distribuição de avaliação média muito dispersa, variando desde 0,5 até 5.

Quando olhamos apenas para os filmes com mais de 50 avaliações, as avaliações médias se reduzem a um range que varia de 2.2 a 4.4, e podemos perceber pelo gráfico também que os filmes com mais avaliações estão no canto superior direito, mostrando que eles tendem a ter uma avaliação média maior.

Com isso concluímos que filmes com médias extremamente altas, ou baixas, possuem poucas avaliações, e filmes com muitas avaliações tendem a ter uma média melhor.

In [ ]:
# Plota a média de avaliação x quantidade de avaliações dos filmes nos nossos datasets
fig = px.scatter(df_movies, x="media_ratings", y="ratings_count",hover_data=['media_ratings'])

fig.update_layout(
    title='Quantidade de avaliações por Média de avaliação',
    xaxis_title="Média de Avaliação",
    yaxis_title="Quantidade de Avaliações",
        template='plotly_white')

fig.show()

- #### Pergunta 10 - Quais foram os filmes que mais geraram engajamento (soma das avaliações e tags atribuídas)? E como estão distribuídas as avaliações destes filmes?

R:

Dentre os 20 filmes que mais geraram engajamento quanto ao número de avaliações e/ou tags atribuídas, nós trouxemos abaixo os gráficos: da quantidade de interações geradas por cada um destes filmes, os boxplots dos ratings de cada um desses filmes, e o Rating Médio dos mesmos.

Destes 20 filmes, apenas 3 não são "inquestionavelmente muito bons!!", são eles:

    - Jurassic Park (1993), que está 9º entre os filmes que geraram maior engajamento (239 interações), mas possui média de avaliações de apenas 3.8 (18º nesta lista) e mediana nas avaliações recebidas em 4.
    - Independence Day (a. k. a. ID4) (1996), que está em 20º entre os filmes que geraram maior engajamento (203 interações), mas possui média de avaliações de apenas 3.4 (20º nesta lista) e mediana nas avaliações recebidas em 3.5.
    - Apollo 13 (1995), que está em 19º entre os filmes que geraram maior engajamento (204 interações), mas possui média de avaliações de apenas 3.8 (19º nesta lista) e mediana nas avaliações recebidas em 4.

Não que estes filmes são sejam bons, mas ficam um pouco atrás dos outros 17, que possuem média de ratings muito boas, dispersão de ratings, em geral, baixa (sendo que muitos destes filmes concentram mais de 75% dos ratings recebidos em pelo menos 4), e grande número de interações geradas (avaliações recebidas ou tags atribuídas).

Com isso, concluímos que filmes que geram muito engajamento, seja por avaliações recebidas, ou tags atribuídas por usuários, não são aclamados à toa, e tendem a distribuição de ratings muito boa, e avaliação média alta.

In [ ]:
# Computando o total de interações/engajamento de cada filme: ratings recebidas + tags atribuídas
df_movies['interacoes_totais'] = df_movies['ratings_count'].replace(np.nan, 0) + df_movies['tags_count'].replace(np.nan, 0)

# ordenando os filmes considerando a quantidade de interações (tags + ratings) em ordem decrescente
df_temp = df_movies.sort_values(by='interacoes_totais',ascending=False)

df_temp = df_temp [0:20]

#Plota o gráfico de total de interações (tags + avaliações) para os 20 primeiros filmes da lista
fig = go.Figure(data=[go.Bar(
            x=df_temp.title, y=df_temp['interacoes_totais'],
            text=df_temp['interacoes_totais'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Top 20 Filmes com maior número de interações',
    xaxis_title="Filme",
    yaxis_title="Quantidade de Interações (Ratings + Tags atribuídas)",
        template='plotly_white')

fig.show()

In [ ]:
# Busca a informação de quantidade de interações (atribuição de rating e/ou tag)
df_ratings = pd.merge(df_ratings, df_movies[['movieId','interacoes_totais']], how = 'left', on=['movieId'])

# Filtra apenas os 20 primeiros filmes com maior engajamento
temp = df_ratings.loc[(df_ratings['interacoes_totais'] >= 203)]

# Plota boxplots das avaliações considerandoos 20 filmes com maior interação
fig = px.box(temp, x="title", y="rating")

# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title="Distribuição das avaliações para os 20 filmes com maior número de interações (Seja por rating ou por tag atribuída)",
    xaxis_title="Título do filme",
    yaxis_title="Distribuição de Ratings",
    template='plotly_white')
    
fig.show()

In [ ]:
# Refazendo os top 20 filmes com melhor rating médio, considerando apenas os 20 filmes com maior número de interações
df_temp = df_movies[df_movies['interacoes_totais'] >= 203].sort_values(by='media_ratings',ascending=False)

df_temp = df_temp[0:20]

#Plota o gráfico de avaliação média dos 20 filmes com maior engajamento
fig = go.Figure(data=[go.Bar(
            x=df_temp.title, y=df_temp['media_ratings'],
            text=df_temp['media_ratings'],
            textposition='auto',
            marker_color='cornflowerblue',
        )])
# Adiciona legenda e titulo dos eixos
fig.update_layout(
    title='Top 20 Filmes com maior engajamento/interação (seja por avaliação ou atribuição de tags)',
    xaxis_title="Título do Filme",
    yaxis_title="Rating Médio",
        template='plotly_white')

fig.show()